# Imports


In [ ]:
!pip install -r requirements.txt
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import cv2
import random
import glob
import seaborn as sns
from inference.py import *
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.losses import JaccardLoss
from segmentation_models_pytorch.utils.metrics import IoU
import warnings
import networkx as nx
import os
from sklearn.metrics import precision_score, recall_score, f1_score
warnings.filterwarnings("ignore")
import pandas as pd
pl.seed_everything(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import rasterio
from aeronet_vector import FeatureCollection, Feature
import shapely
import numpy as np
from matplotlib import pyplot as plt
import os
from dlutils.data import markup_generation, fcutils, angleutils, heightutils, markup_generation
from dlutils.utils import visualization, npfile_utils
import cv2 
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
from inference.py import *
from stability_metrics.py import *

## Prepare data train_test_split

In [63]:
img_paths = sorted(glob.glob('samples_new\\*\images\*.tif'))
Path(img_paths[0]).stem

np.random.seed(0)
random.seed(0)
from tqdm.auto import trange
img_paths = sorted(glob.glob('samples_new\\*\images\*.tif'))
mask_paths = sorted(glob.glob('samples_new\\*\masks\*.tif'))

for i in trange(len(img_paths)):

    mask = Image.open(mask_paths[i])
    if ToTensor()(mask).sum() > w * h * 0.05:
        
        w, h = img.size
        img = Image.open(img_paths[i])

        if np.random.rand() < 0.8:
            dataset = 'train'
        elif np.random.rand() < 0.5:
            dataset = 'val'
        else:
            dataset = 'test'

        save_image(ToTensor()(img), f'samples_cheater/{dataset}/images/' + Path(img_paths[i]).name)
        save_image(ToTensor()(mask), f'samples_cheater/{dataset}/masks/' + Path(mask_paths[i]).name)

for dataset in ['train','val','test']:
    masks_len = len(glob.glob(f'samples_cheater/{dataset}/masks/*.tif'))
    imgs_len =  len(glob.glob(f'samples_cheater/{dataset}/images/*.tif'))
    print(f"{dataset} contains {imgs_len} images and {masks_len} masks")

100%|██████████| 20114/20114 [21:12<00:00, 15.81it/s]


train contains 12460 images and 12460 masks
val contains 1614 images and 1614 masks
test contains 1518 images and 1518 masks


In [ ]:
def pad_if_needed(img, sample_size):
    old_size = img.size

    x_size = (int(old_size[0] / sample_size[0]) + 1)*sample_size[0] if old_size[0] % sample_size[0] != 0 else old_size[0]
    y_size = (int(old_size[1] / sample_size[1]) + 1)*sample_size[1] if old_size[1] % sample_size[1] != 0 else old_size[1]
    new_size = (x_size, y_size) 

    if new_size == old_size:
        return img
    new_img = Image.new(img.mode, new_size, 0)
    new_img.paste(img, (int((new_size[0]-old_size[0])/2),
                        int((new_size[1]-old_size[1])/2)))
        
    return new_img

def crop_samples(img, sample_size):
    
    img = pad_if_needed(img, sample_size)
    x_samples = img.size[0]//sample_size[0]
    y_samples = img.size[1]//sample_size[1]

    samples = []
    ss = sample_size
    for i in range(x_samples):
        for j in range(y_samples):
            sample = img.crop((i*ss[0], j*ss[0], (i+1)*ss[0], (j+1)*ss[1]))

            samples.append(np.array(sample, dtype=np.float32)) # сделали dtype=np.float32 чтобы маски из булевого типа перевести во флоат

    return samples

sample_size = (512,512)

img_paths = sorted(glob.glob('samples\***\\rgb.tif'))
mask_paths = sorted(glob.glob('samples\***\900.tif'))

for i in range(len(img_paths)):

    images, masks = [], []

    img = Image.open(img_paths[i])
    mask = Image.open(mask_paths[i])


    images.extend(crop_samples(img,sample_size))
    masks.extend(crop_samples(mask,sample_size))

    
    for j in range(len(images)):
        
        if masks[j].sum() != 0:  # проверка перед сохранением на пустую маску только для теста и вала
            print(img_paths[i][15:16]+ '_'+ str(j+1)) # 9:11 20:22 15:17
            print(ToTensor()(masks[j]).shape, ToTensor()(images[j]/255).shape)
            save_image(ToTensor()(images[j]/255), 'samples_new/train/images/' + img_paths[i][15:17]+ '_'+ str(j+1) + '.tif')
            save_image(ToTensor()(masks[j]), 'samples_new/train/masks/' + mask_paths[i][15:17] + '_'+ str(j+1) + '.tif')

        

# Dataset

In [11]:
mean = [0.253317  , 0.26740879, 0.23025433]
std  = [0.15158384, 0.14880167, 0.14123519]

train_augs = A.Compose([
    # A.Flip(p=0.5),
    # A.Rotate(border_mode=0, p=0.8),
    # A.RandomScale(scale_limit=0.2, p=0.5),
    A.RandomBrightness(limit=0.2, p=0.2),
    A.RandomContrast(limit=0.2, p=0.2),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.2),
    A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
    A.PadIfNeeded(512,512, border_mode=0),    
    A.CenterCrop(512,512, always_apply=True),
    ToTensorV2()
])

val_augs = A.Compose([
    A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
])

# Train

In [7]:
CFG = {
    'SAVE_DIR': 'E:/diplom_dataset/samples_cheater/',
    'run_name': str(datetime.now()),
    'project_name': 'unet_resnet34_2024',
    'augs_type' : 'color_augs',
    'epochs': 20, 
    'sample_size': (512,512),
    'batch_size': 8,
    'num_workers': 0, #2
    # 'num_samples': 1500,
    'lr': 5e-5,#, 1e-4,
    'use_tpu': False,
    }

In [ ]:
dm = PlDataModule(data_dir = CFG['SAVE_DIR'],
                  train_aug=train_augs, 
                  val_aug=val_augs,
                  batch_size=CFG['batch_size'], #1
                  num_workers=CFG['num_workers'], 
                  use_tpu=CFG['use_tpu'], 
                  train_dset_mode='full', 
                  cpAug=False)

dm.setup()

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

len(train_loader), len(val_loader)

In [ ]:
unet = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet")

#model_name = 'unet34_20_8_0_5e-05_w_strct_augs'
#model = ModelPl.load_from_checkpoint('unet_resnet34/'+ model_name +'.ckpt')

model = ModelPl(unet, JaccardLoss(), lr=CFG['lr'])

save_path = CFG['SAVE_DIR']+CFG['run_name']

#  pip install -U 'wandb>=0.12.10'
wandb_logger = pl.loggers.WandbLogger(name=CFG['run_name']+'_'+str(CFG['epochs'])+'_'+str(CFG['batch_size']), 
                                      project=CFG['project_name'],
                                      config = CFG,
                                      config_exclude_keys = ['SAVE_DIR','run_name''project_name','use_tpu']
                                      #group = 'with_aug',
                                      )
callback = pl.callbacks.ModelCheckpoint(monitor='val IoU',save_top_k=1, 
                                        # dirpath=CFG['SAVE_DIR'],
                                        #filename=CFG['run_name'],
                                          mode='max')
#wandb.config(CFG)
trainer = pl.Trainer(#gpus = 1, 
                     #precision = 16,
                     accelerator='gpu',
                     max_epochs = CFG['epochs'], 
                     logger = wandb_logger,
                     enable_progress_bar=True,
                     #progress_bar_refresh_rate = 2,
                     # callbacks = callback,
                     #callbacks = [TQDMProgressBar(refresh_rate=1)],
                     num_sanity_val_steps=0
                     #accumulate_grad_batches=2,
                     )


In [ ]:
trainer.fit(model, train_loader, val_loader)
trainer.save_checkpoint('unet_resnet34/unet34_'+ str(CFG['epochs']) + '_'+ str(CFG['batch_size']) + '_'+ str(CFG['num_workers']) +'_'+ str(CFG['lr']) +'_w_less_color_augs.ckpt')